# Experiments 19-22 (640 px)

- **Model:**
    1. `yolov8m` *(Medium)*
    2. `yolov9m` *(Medium)*
    3. `yolov11m` *(Medium)*
    4. `yolov12m` *(Medium)*
- **Dataset:** 3.5m | 90º
- **Crop Tiles:** 640px
- **Sizes:** small & mid
- **Experiments:**
    1. YOLO v8 *(Medium)*
    2. YOLO v9 *(Medium)*
    3. YOLO v11 *(Medium)*
    4. YOLO v12 *(Medium)*

Se ha decidido no testear la versión 10 dado que su mayor implementación es la velocidad en el tiempo de inferencia respecto a YOLOv9, pensado en aplicaciones de tiempo real (para detección en video).

## Init

In [5]:
import os
import shutil
import fnmatch
import pickle

In [6]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.2/922.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

## Helper Functions

In [7]:
# Change for different file formats
reference = {
  "small": {
    "suffix": ".S",
    "file": "209"
  },
  "mid": {
    "suffix": ".M",
    "file": "503"
  },
  "large": {
    "suffix": ".L",
    "file": "000"
  }
}

In [8]:
# Clone config files
def copy_config(src_folder, dest_folder):
    """
    Copies files from src_folder to dest_folder, excluding subfolders.

    Args:
        src_folder: The path to the source folder.
        dest_folder: The path to the destination folder.
    """

    try:
        # Ensure destination folder exists
        os.makedirs(dest_folder, exist_ok=True)

        for filename in os.listdir(src_folder):
            src_path = os.path.join(src_folder, filename)
            dest_path = os.path.join(dest_folder, filename)

            if os.path.isfile(src_path):
                shutil.copy2(src_path, dest_path) #copy metadata as well.
                #Use shutil.copy for not copying metadata.
                print(f"Copied: {filename}")
            #else: #optional
                #print(f"Skipped (not a file): {filename}") #optional. Uncomment if you want to see the skipped folders.

        print("✅ Copying complete.")

    except Exception as e:
        print(f"❌ An error occurred: {e}")


In [9]:
# Copy filtered dataset images/labels
def copy_and_filter_folder(src_folder, dest_folder, pattern):
    """
    Copies a folder and files that match the given pattern.
    Alerts the user when a folder or file already exists but *does not* overwrite.
    Creates only what is needed.

    :param src_folder: Path to the source folder.
    :param dest_folder: Path to the destination folder.
    :param pattern: Filename pattern to keep (e.g., "*.txt").
    """
    try:
        # Ensure destination folder exists
        if not os.path.exists(dest_folder):
            print(f"✓ Creating destination folder '{dest_folder}'.\n")
            os.makedirs(dest_folder)
        else:
            print(f"✓ Destination folder '{dest_folder}' already exists.\n")

        # Walk through the source folder
        for root, _, files in os.walk(src_folder):
            relative_path = os.path.relpath(root, src_folder)
            new_root = os.path.join(dest_folder, relative_path)

            if not os.path.exists(new_root):
                print(f"Creating subdirectory '{new_root}'")
                os.makedirs(new_root)
            else:
                print(f"❕Subdirectory '{new_root}' already exists.")
                print("Make sure the data inside is relevant. Otherwise, just delete the folder and repeat the cloning process.")

            for file in files:
                if fnmatch.fnmatch(file, pattern + "*"):
                    src_file = os.path.join(root, file)
                    dest_file = os.path.join(new_root, file)

                    if not os.path.exists(dest_file):
                        shutil.copy2(src_file, dest_file)  # copy metadata as well
                    else:
                        print(f"❗️File '{dest_file}' already exists. Skipping.")

            print(f" ✓ Copying files complete.\n")
        print("✅ Copying dataset complete.")

    except Exception as e:
        print(f"❌ An error occurred: {e}")

In [10]:
def copy_directory(source, destination, overwrite=True):
    try:
        if overwrite and os.path.exists(destination):
            shutil.rmtree(destination)  # Remove existing directory
        shutil.copytree(source, destination)
        print(f"Directory '{source}' copied to '{destination}'")
        return True  # Indicate success
    except FileExistsError:
        print(f"Destination '{destination}' exists. Use overwrite=True to replace.")
        return False  # Indicate failure

In [11]:
# Load/Download prediction results (BB + confidence) as .pkl file

def save_results(results, filename):
    with open(filename, 'wb') as f:
        pickle.dump(results, f)

def load_results(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

In [12]:
def save_on_cloud(source: str, destination: str):
    """
    Saves a folder to a cloud storage location (e.g., Google Drive in Colab).

    Args:
        source (str): The path to the source folder.
        destination (str): The path to the destination folder (in cloud storage).
    """
    # 0. Input Validation (Assertions)
    assert isinstance(source, str), "Source must be a string."
    assert isinstance(destination, str), "Destination must be a string."

    try:
        # 1. Verify Source Folder
        if not os.path.exists(destination):
            os.makedirs(destination)

        # 2. Copy the Folder
        shutil.copytree(source, destination, dirs_exist_ok=True)
        print("✅ Folder copied successfully:\n  ",source,"\n  -->",destination)

    except Exception as e:
        print(f"❌ An error occurred: {e}")

# Datasets builder

## Importing from Drive

In [2]:
!rm -rf /content/sample_data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Check if the cloud path is ok and the dataset can be found
!ls /content/drive/MyDrive/YOLO

3.5m.v3i.yolov8  3.5m.v3i.yolov8.640px	3m60.v2i.yolov8  5m60.v1i.yolov8  5m.v1i.yolov8


In [13]:
drive_path = '/content/drive/MyDrive/YOLO'
drive_datasets_paths = os.listdir(drive_path)
drive_datasets = len(drive_datasets_paths)
if (drive_datasets) > 1:
    print("There are %d dataset options:" % drive_datasets)
else:
    print("Theres is only 1 dataset:")
drive_datasets_paths

Theres is only 1 dataset:


['3.5m.v3i.yolov8.640px']

In [17]:
choose_dataset = 1
index = choose_dataset - 1
model = os.listdir(drive_path)[index]
print("Chosen model:", model)

Chosen model: 3.5m.v3i.yolov8.640px


***Readme:***
*   **Option 1:** is desirable if you need to test many subset combinations in the same session (avoid downloading data twice from the cloud).
*   **Option 2:** is desired if you are goint to work just with one dataset  (avoid downloading unnecessary data from the cloud).
*   **Option 3:** is best if you're just going to test one subset combination  (avoid downloading any data from the cloud).



In [ ]:
# Option 1 (download all datasets)
!cp -r /content/drive/MyDrive/YOLO/ /content
src_folder = f"/content/YOLO/{model}"

In [18]:
# Option 2 (download just the needed dataset)
cloud_path = f"/content/drive/MyDrive/YOLO/{model}/"
local_path = f"/content/YOLO/"
!mkdir $local_path
!cp -r $cloud_path $local_path
src_folder = f"/content/YOLO/{model}"

In [ ]:
# Option 3 (don't download it, just open it from the cloud)
src_folder = f"{drive_path}/{model}"

# Model constructor
1. Model 0: `yolov8m` *(Referencia)*
2. Model 1: `yolov9m`
3. Model 2: `yolov11m`
4. Model 3: `yolov12m`

## Download models

In [ ]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# Load the YOLO model v8
model0 = YOLO("yolov8m.pt")

100%|██████████| 49.7M/49.7M [00:00<00:00, 98.8MB/s]


In [ ]:
# Load the YOLO model v9
model1 = YOLO("yolov9m.pt")

100%|██████████| 39.1M/39.1M [00:01<00:00, 30.6MB/s]


In [ ]:
# Load the YOLO model v11
model2 = YOLO("yolo11m.pt")

100%|██████████| 38.8M/38.8M [00:00<00:00, 48.2MB/s]


In [ ]:
# Load the YOLO model v12
model3 = YOLO("yolo12m.pt")

100%|██████████| 39.0M/39.0M [00:00<00:00, 454MB/s]


# Finetuning

### Info

In [ ]:
!nvidia-smi

Fri Mar  7 20:01:31 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!yolo version

8.3.79


-----
## Experiment 19 (Original)
### *YOLOv8 Medium | ~1000 epochs*

### Train

In [ ]:
# Train Nano model
model0.train(
    data="/content/YOLO/3.5m.v3i.yolov8.640px/data.yaml",
    epochs=1000,
    imgsz=640,
    freeze=10,
    batch=64,
    patience=100
)

Ultralytics 8.3.85 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/YOLO/3.5m.v3i.yolov8.640px/data.yaml, epochs=1000, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=10, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_co

100%|██████████| 755k/755k [00:00<00:00, 25.7MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 114MB/s]


AMP: checks passed ✅


...

                   all        108       2409      0.528      0.482      0.439       0.14
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 290, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



390 epochs completed in 0.838 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 52.1MB
Optimizer stripped from runs/detect/train/weights/best.pt, 52.1MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.85 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:02<00:00,  2.27s/it]


                   all        108       2409      0.522       0.48      0.446      0.148
Speed: 0.2ms preprocess, 10.8ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e9761e43510>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

### Validation

In [ ]:
# Load currently trained YOLO model
model0 = YOLO("/content/runs/detect/train/weights/best.pt")

In [ ]:
# Load stored model
# model1 = YOLO("/content/drive/MyDrive/save/detect/train/weights/best.pt")

In [ ]:
# Validate the model
model0.val(data="/content/YOLO/3.5m.v3i.yolov8.640px/data.yaml")

Ultralytics 8.3.85 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning /content/YOLO/3.5m.v3i.yolov8.640px/valid/labels.cache... 108 images, 0 backgrounds, 0 corrupt: 100%|██████████| 108/108 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:08<00:00,  1.15s/it]


                   all        108       2409      0.521      0.481      0.446      0.147
Speed: 4.8ms preprocess, 25.7ms inference, 0.0ms loss, 10.8ms postprocess per image
Results saved to runs/detect/val


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e976aac51d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

### Save results

In [ ]:
# Store model weights and metrics
save_on_cloud(source='/content/runs/', destination='/content/drive/MyDrive/save/')

✅ Folder copied successfully:
   /content/runs/ 
  --> /content/drive/MyDrive/save/


-----
## Experiment 20
### *YOLOv9 Medium | ~1000 epochs*

### Train

In [ ]:
# Train Nano model
model1.train(
    data="/content/YOLO/3.5m.v3i.yolov8.640px/data.yaml",
    epochs=1000,
    imgsz=640,
    freeze=10,
    batch=64,
    patience=100
)

Ultralytics 8.3.85 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov9m.pt, data=/content/YOLO/3.5m.v3i.yolov8.640px/data.yaml, epochs=1000, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=10, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_co

100%|██████████| 755k/755k [00:00<00:00, 76.6MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1    171648  ultralytics.nn.modules.block.RepNCSPELAN4    [64, 128, 128, 64, 1]         
  3                  -1  1    276960  ultralytics.nn.modules.block.AConv           [128, 240]                    
  4                  -1  1    629520  ultralytics.nn.modules.block.RepNCSPELAN4    [240, 240, 240, 120, 1]       
  5                  -1  1    778320  ultralytics.nn.modules.block.AConv           [240, 360]                    
  6                  -1  1   1414080  ultralytics.nn.modules.block.RepNCSPELAN4    [360, 360, 360, 180, 1]       
  7                  -1  1   1556160  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 391MB/s]


AMP: checks passed ✅


...

                   all        108       2409      0.533      0.476      0.442      0.143
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 180, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



280 epochs completed in 0.748 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 40.9MB
Optimizer stripped from runs/detect/train/weights/best.pt, 40.9MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.85 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv9m summary (fused): 151 layers, 20,013,715 parameters, 0 gradients, 76.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:02<00:00,  2.26s/it]


                   all        108       2409      0.547      0.487      0.463      0.155
Speed: 0.2ms preprocess, 13.7ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fc881fed590>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

### Validation

In [ ]:
# Load currently trained YOLO model
model1 = YOLO("/content/runs/detect/train/weights/best.pt")

In [ ]:
# Load stored model
# model1 = YOLO("/content/drive/MyDrive/save/detect/train/weights/best.pt")

In [ ]:
# Validate the model
model1.val(data="/content/YOLO/3.5m.v3i.yolov8.640px/data.yaml")

Ultralytics 8.3.85 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv9m summary (fused): 151 layers, 20,013,715 parameters, 0 gradients, 76.5 GFLOPs


val: Scanning /content/YOLO/3.5m.v3i.yolov8.640px/valid/labels.cache... 108 images, 0 backgrounds, 0 corrupt: 100%|██████████| 108/108 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:08<00:00,  1.26s/it]


                   all        108       2409      0.545      0.486      0.462      0.154
Speed: 3.6ms preprocess, 34.3ms inference, 0.0ms loss, 6.0ms postprocess per image
Results saved to runs/detect/val


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fc96ae57850>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

### Save results

In [ ]:
# Store model weights and metrics
save_on_cloud(source='/content/runs/', destination='/content/drive/MyDrive/save1/')

✅ Folder copied successfully:
   /content/runs/ 
  --> /content/drive/MyDrive/save/


-----
## Experiment 21
### *YOLOv11 Medium | ~1000 epochs*

### Train

La memoria de la GPU gratuita de Collab no es suficiente para ejecutar con batch size 64. (`OutOfMemoryError`)

In [ ]:
# Train Nano model
model2.train(
    data="/content/YOLO/3.5m.v3i.yolov8.640px/data.yaml",
    epochs=1000,
    imgsz=640,
    freeze=10,
    batch=16, # Batch size reducido (no corría con 32 ni 64)
    patience=100
)

Ultralytics 8.3.85 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11m.pt, data=/content/YOLO/3.5m.v3i.yolov8.640px/data.yaml, epochs=1000, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=10, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_co

100%|██████████| 755k/755k [00:00<00:00, 34.6MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    111872  ultralytics.nn.modules.block.C3k2            [128, 256, 1, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  1    444928  ultralytics.nn.modules.block.C3k2            [256, 512, 1, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  1   1380352  ultralytics.nn.modules.block.C3k2            [512, 512, 1, True]           
  7                  -1  1   2360320  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 145MB/s]


AMP: checks passed ✅


...

                   all        108       2409      0.533        0.5      0.464      0.143
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 79, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



179 epochs completed in 0.424 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 40.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 40.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.85 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11m summary (fused): 125 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:06<00:00,  1.54s/it]


                   all        108       2409      0.518       0.52      0.466      0.157
Speed: 1.0ms preprocess, 12.9ms inference, 0.0ms loss, 5.0ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f89db065f90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

### Validation

In [ ]:
# Load currently trained YOLO model
model2 = YOLO("/content/runs/detect/train/weights/best.pt")

In [ ]:
# Load stored model
# model1 = YOLO("/content/drive/MyDrive/save/detect/train/weights/best.pt")

In [ ]:
# Validate the model
model2.val(data="/content/YOLO/3.5m.v3i.yolov8.640px/data.yaml")

Ultralytics 8.3.85 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11m summary (fused): 125 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs


val: Scanning /content/YOLO/3.5m.v3i.yolov8.640px/valid/labels.cache... 108 images, 0 backgrounds, 0 corrupt: 100%|██████████| 108/108 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:08<00:00,  1.24s/it]


                   all        108       2409       0.52      0.522      0.469      0.156
Speed: 3.1ms preprocess, 26.2ms inference, 0.0ms loss, 11.4ms postprocess per image
Results saved to runs/detect/val


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f8aba7f7250>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

### Save results

In [ ]:
# Store model weights and metrics
save_on_cloud(source='/content/runs/', destination='/content/drive/MyDrive/save2/')

✅ Folder copied successfully:
   /content/runs/ 
  --> /content/drive/MyDrive/save2/


-----
## Experiment 22
### *YOLOv12 Medium | ~1000 epochs*

### Train

In [ ]:
# Train Nano model
model3.train(
    data="/content/YOLO/3.5m.v3i.yolov8.640px/data.yaml",
    epochs=1000,
    imgsz=640,
    freeze=10,
    batch=64,
    patience=100
)

Ultralytics 8.3.85 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo12m.pt, data=/content/YOLO/3.5m.v3i.yolov8.640px/data.yaml, epochs=1000, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=10, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_co

100%|██████████| 755k/755k [00:00<00:00, 24.9MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    111872  ultralytics.nn.modules.block.C3k2            [128, 256, 1, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  1    444928  ultralytics.nn.modules.block.C3k2            [256, 512, 1, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  2   2689536  ultralytics.nn.modules.block.A2C2f           [512, 512, 2, True, 4]        
  7                  -1  1   2360320  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 123MB/s]


AMP: checks passed ✅


...

                   all        108       2409      0.527      0.483      0.463      0.153
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 98, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



198 epochs completed in 0.563 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 40.8MB
Optimizer stripped from runs/detect/train/weights/best.pt, 40.8MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.85 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv12m summary (fused): 169 layers, 20,105,683 parameters, 0 gradients, 67.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:02<00:00,  2.64s/it]


                   all        108       2409       0.52      0.528      0.489      0.168
Speed: 0.2ms preprocess, 17.3ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7bd1633c1150>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

### Validation

In [ ]:
# Load currently trained YOLO model
model3 = YOLO("/content/runs/detect/train/weights/best.pt")

In [ ]:
# Load stored model
# model1 = YOLO("/content/drive/MyDrive/save/detect/train/weights/best.pt")

In [ ]:
# Validate the model
model3.val(data="/content/YOLO/3.5m.v3i.yolov8.640px/data.yaml")

Ultralytics 8.3.85 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv12m summary (fused): 169 layers, 20,105,683 parameters, 0 gradients, 67.1 GFLOPs


val: Scanning /content/YOLO/3.5m.v3i.yolov8.640px/valid/labels.cache... 108 images, 0 backgrounds, 0 corrupt: 100%|██████████| 108/108 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:11<00:00,  1.59s/it]


                   all        108       2409      0.514      0.523      0.486      0.167
Speed: 7.8ms preprocess, 35.1ms inference, 0.0ms loss, 11.9ms postprocess per image
Results saved to runs/detect/val


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7bd1632c2e90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

### Save results

In [ ]:
# Store model weights and metrics
save_on_cloud(source='/content/runs/', destination='/content/drive/MyDrive/save3/')

✅ Folder copied successfully:
   /content/runs/ 
  --> /content/drive/MyDrive/save3/


# Conclusions

| Metrics           | Experiment 1 _(Small & Medium)_ | Experiment 2 _(Small)_ | Experiment 3 _(Medium)_ |
|-----------------|--------------------------------|-----------------------|------------------------|
| **mAP50-95(B)** | 0.                          |                  |  |
| **Precision** | ~0.                           |          |           |
| **Recall** | ~0.                           |         |          |
| **Observations**|   |   |   |

1. **Experiment 1: _small & medium plant sizes_**

- **mAP50-95(B):**
- **Precision:**  
- **Recall:**  
- **Observations:**  

2. **Experiment 2: _small plant sizes_**

- **mAP50-95(B):**  
- **Precision:**  
- **Recall:**  
- **Observations:**  

3. **Experiment 3: _medium plant sizes_**

- **mAP50-95(B):**  
- **Precision:**  
- **Recall:**  
- **Observations:**  

#### Final Conclusions
-